# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("output.csv")
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
0,jamestown,0,AU,1569879684,46,-33.21,138.60,138.60,285.70
1,tasiilaq,75,GL,1569879793,52,65.61,-37.64,-37.64,279.15
2,taolanaro,75,GL,1569879793,52,65.61,-37.64,-37.64,279.15
3,tuatapere,29,NZ,1569879789,77,-46.13,167.69,167.69,279.82
4,victoria,90,BN,1569879774,94,5.28,115.24,115.24,297.04
5,yar-sale,100,RU,1569879824,97,66.83,70.83,70.83,276.00
6,cardston,1,CA,1569880075,74,49.20,-113.32,-113.32,270.93
7,codrington,75,AU,1569879759,72,-28.95,153.24,153.24,290.15
8,los llanos de aridane,40,ES,1569879797,78,28.66,-17.92,-17.92,294.82
9,kazalinsk,40,ES,1569879797,78,28.66,-17.92,-17.92,294.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coordinates = df[["Lat", "Lng"]]
humidity = df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=10,point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df=df[["City","Cloudiness","Humidity","Max Temp", "Wind_Speed"]]
new_df.dropna()
new_df

,City,Cloudiness,Humidity,Max Temp,Wind_Speed
0,jamestown,0,46,138.60,285.70
1,tasiilaq,75,52,-37.64,279.15
2,taolanaro,75,52,-37.64,279.15
3,tuatapere,29,77,167.69,279.82
4,victoria,90,94,115.24,297.04
5,yar-sale,100,97,70.83,276.00
6,cardston,1,74,-113.32,270.93
7,codrington,75,72,153.24,290.15
8,los llanos de aridane,40,78,-17.92,294.82
9,kazalinsk,40,78,-17.92,294.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
Hotel_name=[]
for i in range(len(hotel_df["City"])):
    params = {
        "location": [hotel_df["Lat"][i],hotel_df["Lng"][i]],
        "rankby": "distance",
        "type": "hotel",
        "key": g_key,
    }
    try:
        response = requests.get(base_url, params=params).json()
        results = response['results'][0]["name"]
        Hotel_name.append(results)
    except IndexError:
        Hotel_name.append('')
        

In [ ]:
hotel_df["Hotel Name"]=Hotel_name

In [ ]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=10,point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Display Map